In [2]:
import pandas as pd
import numpy as np

In [3]:
# Remove any line wrapping limit
pd.set_option('display.width', 1000)

In [4]:
# Uploading the data
df = pd.read_excel("Loss data.xlsx")

# info 
print(df.head())

   Accident Month  Report Month   Total Loss
0          202101        202112  1083.000000
1          202102        202112  1019.000000
2          202103        202112   912.000000
3          202104        202112  1026.151930
4          202105        202112  1030.272953


In [5]:
# Converting the first two Columns to datetime format
df["Accident Month"] = pd.to_datetime(df["Accident Month"], format ="%Y%m")
df["Report Month"] = pd.to_datetime(df["Report Month"], format="%Y%m")


In [6]:
# Calcualting Development year
df["Development Month"] = ((df["Report Month"].dt.year - df["Accident Month"].dt.year) * 12 + (df["Report Month"].dt.month - df["Accident Month"].dt.month))

In [7]:
triangle = df.pivot(index="Accident Month", columns="Development Month", values="Total Loss")

triangle = triangle.sort_index().round(2)

print("Loss Triangle:")
print(triangle.head(12))

Loss Triangle:
Development Month     0       1      2       3        4        5        6        7        8       9       10      11
Accident Month                                                                                                      
2021-01-01         494.5  646.67  791.2  919.64  1004.72  1031.07  1000.99   966.75  1047.07  1058.0  1044.0  1083.0
2021-02-01         466.5  664.67  792.0  809.82   988.68   885.44  1038.42   900.25   930.51  1076.0  1019.0     NaN
2021-03-01         501.0  705.33  760.8  816.07   975.47  1051.46  1031.53   934.99  1026.15   912.0     NaN     NaN
2021-04-01         450.0  696.67  732.0  897.32   879.25   895.15   960.59   968.73  1026.15     NaN     NaN     NaN
2021-05-01         458.5  674.67  792.8  979.46  1006.60   912.62  1012.81  1030.27      NaN     NaN     NaN     NaN
2021-06-01         476.0  668.00  791.2  903.57   879.25  1064.08   931.03      NaN      NaN     NaN     NaN     NaN
2021-07-01         469.5  681.33  751.2  960.71  

In [8]:
# converting to cumualtive to use chaining lader method
def incremental_to_cumulative(triangle):

  cumulative_triangle = triangle.copy()
  for dev_month in cumulative_triangle.columns[1:]:
    cumulative_triangle[dev_month] = cumulative_triangle[dev_month] + cumulative_triangle[dev_month - 1]
  return cumulative_triangle


cumulative_triangle = incremental_to_cumulative(triangle)
print("\nCumulative Loss Triangle:")
print(cumulative_triangle.head(12))



Cumulative Loss Triangle:
Development Month     0        1        2        3        4        5        6        7        8        9         10        11
Accident Month                                                                                                               
2021-01-01         494.5  1141.17  1932.37  2852.01  3856.73  4887.80  5888.79  6855.54  7902.61  8960.61  10004.61  11087.61
2021-02-01         466.5  1131.17  1923.17  2732.99  3721.67  4607.11  5645.53  6545.78  7476.29  8552.29   9571.29       NaN
2021-03-01         501.0  1206.33  1967.13  2783.20  3758.67  4810.13  5841.66  6776.65  7802.80  8714.80       NaN       NaN
2021-04-01         450.0  1146.67  1878.67  2775.99  3655.24  4550.39  5510.98  6479.71  7505.86      NaN       NaN       NaN
2021-05-01         458.5  1133.17  1925.97  2905.43  3912.03  4824.65  5837.46  6867.73      NaN      NaN       NaN       NaN
2021-06-01         476.0  1144.00  1935.20  2838.77  3718.02  4782.10  5713.13      NaN    

In [9]:
# calcualte the link ratio

link_ratios = []
max_dev = triangle.shape[1]

for col in range(max_dev - 1):
  current_col =triangle.iloc[:, col]
  next_col = triangle.iloc[:, col+1]

  vaild = current_col.notna() & next_col.notna()
  ratio = next_col[vaild].sum() / current_col[vaild].sum()

  link_ratios.append(ratio)

print("\nLink Ratios:")
print([round(float(r), 2) for r in link_ratios])




Link Ratios:
[1.4, 1.12, 1.16, 1.07, 1.0, 1.02, 0.95, 1.07, 1.01, 0.97, 1.04]


In [10]:
# using Chain ladder method

projected_triangle = cumulative_triangle.copy()

for i in range(projected_triangle.shape[0]):
  for j in range(projected_triangle.shape[1]):
    if pd.isna(projected_triangle.iat[i, j]):
      if j == 0:
        continue

      prev_value = projected_triangle.iat[i, j-1]
      projected_value = prev_value * link_ratios[j-1]
      projected_triangle.iat[i, j] = projected_value

print(projected_triangle.round(2))

Development Month     0        1        2        3        4        5        6        7        8        9         10        11
Accident Month                                                                                                               
2021-01-01         494.5  1141.17  1932.37  2852.01  3856.73  4887.80  5888.79  6855.54  7902.61  8960.61  10004.61  11087.61
2021-02-01         466.5  1131.17  1923.17  2732.99  3721.67  4607.11  5645.53  6545.78  7476.29  8552.29   9571.29   9928.84
2021-03-01         501.0  1206.33  1967.13  2783.20  3758.67  4810.13  5841.66  6776.65  7802.80  8714.80   8424.85   8739.57
2021-04-01         450.0  1146.67  1878.67  2775.99  3655.24  4550.39  5510.98  6479.71  7505.86  7611.49   7358.25   7633.12
2021-05-01         458.5  1133.17  1925.97  2905.43  3912.03  4824.65  5837.46  6867.73  7339.75  7443.03   7195.40   7464.19
2021-06-01         476.0  1144.00  1935.20  2838.77  3718.02  4782.10  5713.13  5437.52  5811.23  5893.01   5696.95   

In [11]:
# Estimate IBNR
reported_claims = np.nansum(triangle.values)
ultimate_claims = projected_triangle.values.sum()
ibnr = ultimate_claims - reported_claims

print(f"\nEstimated IBNR: {ibnr:.2f}")


Estimated IBNR: 457406.48
